In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 97 kB 2.4 MB/s             


In [1]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

# 一 简单模型启动
## 1.1 环境安装

In [5]:
# 安装 PaddlePaddle 框架，由于 AIStudio 已配置相应环境，无需安装
# !pip install paddlepaddle==1.8.5 -t /home/aistudio/external-libraries
# !pip install pgl==1.2.1 -t /home/aistudio/external-libraries #安装PGL ,2022.1.1已经安装


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 7.9 MB 11.2 MB/s            
     |████████████████████████████████| 2.7 MB 60.7 MB/s            
     |████████████████████████████████| 42 kB 2.5 MB/s             
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/50/46/292cff79f5b30151b027400efdb3f740ea03271b600751b6696cf550c10d/numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
     |████████████████████████████████| 1.9 MB 58.5 MB/s            
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/2d/61/08076519c80041bc0ffa1a8af0cbd3bf3e2b62af10435d269a9d0f40564d/requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/03/6b/12b307892ef5e9f4a1a92f5996922561c453540f5ce22a09038d2486b451/protobuf-3.19.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d9/5a/e7c31adbe875f2ab

## 1.2 PGL创建图

In [5]:
import pgl
from pgl import graph  # 导入 PGL 中的图模块
import paddle.fluid as fluid # 导入飞桨框架
import numpy as np

def build_graph():
    # 定义图中的节点数目，我们使用数字来表示图中的每个节点
    num_nodes = 10

    # 定义图中的边集，每条边用一个tuple表示
    edge_list = [(2, 0), (2, 1), (3, 1),(4, 0), (5, 0),
             (6, 0), (6, 4), (6, 5), (7, 0), (7, 1),
             (7, 2), (7, 3), (8, 0), (9, 7)]

    # 随机初始化节点特征，特征维度为 d
    d = 16
    feature = np.random.randn(num_nodes, d).astype("float32")

    # 随机地为每条边赋值一个权重
    edge_feature = np.random.randn(len(edge_list), 1).astype("float32")

    # 创建图对象，最多四个输入
    g = graph.Graph(num_nodes = num_nodes, #点的个数
                    edges = edge_list, # 边的列表
                    node_feat = {'feature':feature}, # 节点特征，需要numpy数组格式
                    edge_feat ={'edge_feature': edge_feature}) # 边特征，需要numpy数组格式

    return g


In [23]:
g = build_graph()
print('图中共计 %d 个节点' % g.num_nodes)
print('图中共计 %d 条边' % g.num_edges)
g.edge_feat['edge_feature'].shape # 边特征
print(g.edge_feat['edge_feature'].shape)
print(g.node_feat['feature'].shape)


图中共计 10 个节点
图中共计 14 条边
(14, 1)
(10, 16)


{'feature': array([[-0.2477345 , -0.812736  , -1.4650254 ,  0.4828808 , -0.764734  ,
          0.3886107 ,  0.36288208,  0.26333258,  0.7573494 , -0.5630978 ,
         -2.6487281 ,  1.3338678 ,  0.14907381, -0.35734096,  1.0074781 ,
          0.88632774],
        [ 0.16336803, -1.6093405 , -0.05840158, -0.40328953,  1.2657588 ,
         -1.0222794 ,  0.14264646,  0.13105309, -0.979174  , -1.1595355 ,
          0.12038045,  0.20591302,  0.92540574, -0.4145023 ,  0.50135154,
          0.43452904],
        [ 1.0713071 ,  2.4842212 ,  0.8265732 ,  0.12600829, -0.1596692 ,
          0.48708674,  0.7626266 , -0.49102503, -0.30354235, -0.35510728,
         -0.14856303,  0.64257216, -1.0370559 ,  1.3658324 , -1.0982478 ,
         -0.504533  ],
        [-3.5079772 , -0.21437575, -0.06436482, -0.03339192, -0.56347525,
          0.15562916,  0.91232836,  2.026211  , -0.5154434 ,  0.71511346,
          0.38422295,  2.4751744 ,  1.6493696 ,  0.192638  , -0.7111548 ,
         -0.6326861 ],
        [

## 定义层

In [26]:
# 定义一个同时传递节点特征和边权重的简单模型层。
def model_layer(gw, nfeat, efeat, hidden_size, name, activation):
    '''
    gw: GraphWrapper 图数据容器，用于在定义模型的时候使用，后续训练时再feed入真实数据
    nfeat: 节点特征
    efeat: 边权重
    hidden_size: 模型隐藏层维度
    activation: 使用的激活函数
    '''

    # 定义 send 函数
    def send_func(src_feat, dst_feat, edge_feat):
        # 将源节点的节点特征和边权重共同作为消息发送
        return src_feat['h'] * edge_feat['e']

    # 定义 recv 函数
    def recv_func(feat):
        # 目标节点接收源节点消息，采用 sum 的聚合方式
        return fluid.layers.sequence_pool(feat, pool_type='sum')

    # 触发消息传递机制
    msg = gw.send(send_func, nfeat_list=[('h', nfeat)], efeat_list=[('e', efeat)])
    output = gw.recv(msg, recv_func)
    output = fluid.layers.fc(output,
                    size=hidden_size,
                    bias_attr=False,
                    act=activation,
                    name=name)
    return output

## 定义模型

In [27]:
class Model(object):
    def __init__(self, graph):
        """
        graph: 我们前面创建好的图
        """
        # 创建 GraphWrapper 图数据容器，用于在定义模型的时候使用，后续训练时再feed入真实数据
        self.gw = pgl.graph_wrapper.GraphWrapper(name='graph',
                    node_feat=graph.node_feat_info(),
                    edge_feat=graph.edge_feat_info())
        # 作用同 GraphWrapper，此处用作节点标签的容器
        self.node_label = fluid.layers.data("node_label", shape=[None, 1],
                    dtype="float32", append_batch_size=False)

    def build_model(self):
        # 定义两层model_layer
        output = model_layer(self.gw, 
                             self.gw.node_feat['feature'], 
                             self.gw.edge_feat['edge_feature'],
                             hidden_size=8, 
                             name='layer_1', 
                             activation='relu')
        output = model_layer(self.gw, 
                             output, 
                             self.gw.edge_feat['edge_feature'],
                             hidden_size=1, 
                             name='layer_2', 
                             activation=None)
                             
        # 对于二分类任务，可以使用以下 API 计算损失
        loss = fluid.layers.sigmoid_cross_entropy_with_logits(x=output, 
                                                              label=self.node_label)
        # 计算平均损失
        loss = fluid.layers.mean(loss)
        
        # 计算准确率
        prob = fluid.layers.sigmoid(output)
        pred = prob > 0.5
        pred = fluid.layers.cast(prob > 0.5, dtype="float32")
        correct = fluid.layers.equal(pred, self.node_label)
        correct = fluid.layers.cast(correct, dtype="float32")
        acc = fluid.layers.reduce_mean(correct)

        return loss, acc

## 训练准备

In [28]:
# 是否在 GPU 或 CPU 环境运行
use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

# 定义程序，也就是我们的 Program
startup_program = fluid.Program() # 用于初始化模型参数
train_program = fluid.Program()   # 训练时使用的主程序，包含前向计算和反向梯度计算
test_program = fluid.Program()    # 测试时使用的程序，只包含前向计算

with fluid.program_guard(train_program, startup_program):
    model = Model(g)
    # 创建模型和计算 Loss
    loss, acc = model.build_model()
    # 选择Adam优化器，学习率设置为0.01
    adam = fluid.optimizer.Adam(learning_rate=0.01)
    adam.minimize(loss) # 计算梯度和执行梯度反向传播过程

# 复制构造 test_program，与 train_program的区别在于不需要梯度计算和反向过程。
test_program = train_program.clone(for_test=True)

# 定义一个在 place(CPU)上的Executor来执行program
exe = fluid.Executor(place)
# 参数初始化
exe.run(startup_program) 

# 获取真实图数据
feed_dict = model.gw.to_feed(g) 
# 获取真实标签数据
# 由于我们是做节点分类任务，因此可以简单的用0、1表示节点类别。其中，黄色点标签为0，绿色点标签为1。
y = [0,1,1,1,0,0,0,1,0,1]
label = np.array(y, dtype="float32")
label = np.expand_dims(label, -1)
feed_dict['node_label'] = label

/home/aistudio/external-libraries/pgl/graph_wrapper.py:151: UserWarning: The edge features in argument `efeat_list` should be fetched from a instance of `pgl.graph_wrapper.GraphWrapper`, because we have sorted the edges and the order of edges is changed.
Therefore, if you use external edge features, the order of features of each edge may not match its edge, which can cause serious errors.
If you use the `efeat_list` correctly, please ignore this warning.
  "The edge features in argument `efeat_list` should be fetched "


In [29]:
print(feed_dict)

{'graph/num_edges': array([14]), 'graph/edges_src': array([2, 4, 5, 6, 7, 8, 2, 3, 7, 7, 7, 6, 6, 9]), 'graph/edges_dst': array([0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 5, 7]), 'graph/num_nodes': array([10]), 'graph/uniq_dst': array([0, 1, 2, 3, 4, 5, 7]), 'graph/uniq_dst_count': array([ 0,  6,  9, 10, 11, 12, 13, 14], dtype=int32), 'graph/indegree': array([6, 3, 1, 1, 1, 1, 0, 1, 0, 0]), 'graph/graph_lod': array([ 0, 10], dtype=int32), 'graph/num_graph': array([1]), 'graph/node_feat/feature': array([[-0.2477345 , -0.812736  , -1.4650254 ,  0.4828808 , -0.764734  ,
         0.3886107 ,  0.36288208,  0.26333258,  0.7573494 , -0.5630978 ,
        -2.6487281 ,  1.3338678 ,  0.14907381, -0.35734096,  1.0074781 ,
         0.88632774],
       [ 0.16336803, -1.6093405 , -0.05840158, -0.40328953,  1.2657588 ,
        -1.0222794 ,  0.14264646,  0.13105309, -0.979174  , -1.1595355 ,
         0.12038045,  0.20591302,  0.92540574, -0.4145023 ,  0.50135154,
         0.43452904],
       [ 1.0713071 ,  2

In [30]:
for epoch in range(30):
    train_loss = exe.run(train_program,
        feed=feed_dict,      # feed入真实训练数据
        fetch_list=[loss],   # fetch出需要的计算结果
        return_numpy=True)[0]
    print('Epoch %d | Loss: %f' % (epoch, train_loss))

Epoch 0 | Loss: 1.171933
Epoch 1 | Loss: 1.038481
Epoch 2 | Loss: 0.904573
Epoch 3 | Loss: 0.787878
Epoch 4 | Loss: 0.705820
Epoch 5 | Loss: 0.650643
Epoch 6 | Loss: 0.613303
Epoch 7 | Loss: 0.582065
Epoch 8 | Loss: 0.561654
Epoch 9 | Loss: 0.545847
Epoch 10 | Loss: 0.531641
Epoch 11 | Loss: 0.518906
Epoch 12 | Loss: 0.507505
Epoch 13 | Loss: 0.497302
Epoch 14 | Loss: 0.488169
Epoch 15 | Loss: 0.479996
Epoch 16 | Loss: 0.472690
Epoch 17 | Loss: 0.466169
Epoch 18 | Loss: 0.460366
Epoch 19 | Loss: 0.455218
Epoch 20 | Loss: 0.450667
Epoch 21 | Loss: 0.446660
Epoch 22 | Loss: 0.443143
Epoch 23 | Loss: 0.440067
Epoch 24 | Loss: 0.437382
Epoch 25 | Loss: 0.435043
Epoch 26 | Loss: 0.433008
Epoch 27 | Loss: 0.431238
Epoch 28 | Loss: 0.429698
Epoch 29 | Loss: 0.428358


In [31]:
test_acc = exe.run(test_program, feed=feed_dict, fetch_list=[acc], return_numpy=True)[0]
print("Test Acc: %f" % test_acc)

Test Acc: 0.800000


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:613: UserWarning: The variable graph/edges_dst is not found in program. It is not declared or is pruned.
  % name)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:613: UserWarning: The variable graph/indegree is not found in program. It is not declared or is pruned.
  % name)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:613: UserWarning: The variable graph/graph_lod is not found in program. It is not declared or is pruned.
  % name)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:613: UserWarning: The variable graph/num_graph is not found in program. It is not declared or is pruned.
  % name)


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 